In [1]:
%pylab inline
import pandas as pd
import seaborn as sns
sns.set_style('white')
sns.set_context('paper', font_scale=2)
import pyfaidx
from Bio.Seq import Seq
from Bio.Alphabet import generic_dna

fasta = pyfaidx.Fasta('/home/cmb-panasas2/skchoudh/genomes/hg38/fasta/hg38.fa')

Populating the interactive namespace from numpy and matplotlib


In [2]:
from collections import defaultdict
import gzip
import pandas as pd
import re


GTF_HEADER  = ['seqname', 'source', 'feature', 'start', 'end', 'score',
               'strand', 'frame']
R_SEMICOLON = re.compile(r'\s*;\s*')
R_COMMA     = re.compile(r'\s*,\s*')
R_KEYVALUE  = re.compile(r'(\s+|\s*=\s*)')


def dataframe(filename):
    """Open an optionally gzipped GTF file and return a pandas.DataFrame.
    """
    # Each column is a list stored as a value in this dict.
    result = defaultdict(list)

    for i, line in enumerate(lines(filename)):
        for key in line.keys():
            # This key has not been seen yet, so set it to None for all
            # previous lines.
            if key not in result:
                result[key] = [None] * i

        # Ensure this row has some value for each column.
        for key in result.keys():
            result[key].append(line.get(key, None))

    return pd.DataFrame(result)


def lines(filename):
    """Open an optionally gzipped GTF file and generate a dict for each line.
    """
    fn_open = gzip.open if filename.endswith('.gz') else open

    with fn_open(filename) as fh:
        for line in fh:
            if line.startswith('#'):
                continue
            else:
                yield parse(line)


def parse(line):
    """Parse a single GTF line and return a dict.
    """
    result = {}

    fields = line.rstrip().split('\t')

    for i, col in enumerate(GTF_HEADER):
        result[col] = _get_value(fields[i])

    # INFO field consists of "key1=value;key2=value;...".
    infos = [x for x in re.split(R_SEMICOLON, fields[8]) if x.strip()]

    for i, info in enumerate(infos, 1):
        # It should be key="value".
        try:
            key, _, value = re.split(R_KEYVALUE, info, 1)
        # But sometimes it is just "value".
        except ValueError:
            key = 'INFO{}'.format(i)
            value = info
        # Ignore the field if there is no value.
        if value:
            result[key] = _get_value(value)

    return result


def _get_value(value):
    if not value:
        return None

    # Strip double and single quotes.
    value = value.strip('"\'')

    # Return a list if the value has a comma.
    if ',' in value:
        value = re.split(R_COMMA, value)
    # These values are equivalent to None.
    elif value in ['', '.', 'NA']:
        return None

    return value

def get_seq(row):
    chrom = row['seqname']
    start = row['start']
    end = row['end']
    strand = row['strand']
    
    if strand == '+':
        return fasta.get_seq(chrom, start, end)
    else:
        seq = fasta.get_seq(chrom, start, end)
        return str(Seq(seq, generic_dna).reverse_complement())

In [3]:
gtf = dataframe('/home/cmb-panasas2/skchoudh/genomes/hg38/annotation/gencode.v25.annotation.gtf')
gtf['start'] = gtf['start'].astype(int)
gtf['end'] = gtf['end'].astype(int)

In [4]:
gtf.head()

,ccdsid,end,exon_id,exon_number,feature,frame,gene_id,gene_name,gene_status,gene_type,...,seqname,source,start,strand,tag,transcript_id,transcript_name,transcript_status,transcript_support_level,transcript_type
0,None,14409,None,None,gene,None,ENSG00000223972.5,DDX11L1,KNOWN,transcribed_unprocessed_pseudogene,...,chr1,HAVANA,11869,+,None,None,None,None,None,None
1,None,14409,None,None,transcript,None,ENSG00000223972.5,DDX11L1,KNOWN,transcribed_unprocessed_pseudogene,...,chr1,HAVANA,11869,+,basic,ENST00000456328.2,DDX11L1-002,KNOWN,1,processed_transcript
2,None,12227,ENSE00002234944.1,1,exon,None,ENSG00000223972.5,DDX11L1,KNOWN,transcribed_unprocessed_pseudogene,...,chr1,HAVANA,11869,+,basic,ENST00000456328.2,DDX11L1-002,KNOWN,1,processed_transcript
3,None,12721,ENSE00003582793.1,2,exon,None,ENSG00000223972.5,DDX11L1,KNOWN,transcribed_unprocessed_pseudogene,...,chr1,HAVANA,12613,+,basic,ENST00000456328.2,DDX11L1-002,KNOWN,1,processed_transcript
4,None,14409,ENSE00002312635.1,3,exon,None,ENSG00000223972.5,DDX11L1,KNOWN,transcribed_unprocessed_pseudogene,...,chr1,HAVANA,13221,+,basic,ENST00000456328.2,DDX11L1-002,KNOWN,1,processed_transcript


In [5]:
start_codons = gtf[gtf.feature=='start_codon']
start_codons.head()

,ccdsid,end,exon_id,exon_number,feature,frame,gene_id,gene_name,gene_status,gene_type,...,seqname,source,start,strand,tag,transcript_id,transcript_name,transcript_status,transcript_support_level,transcript_type
56,CCDS30547.1,69093,ENSE00002319515.1,1,start_codon,0,ENSG00000186092.4,OR4F5,KNOWN,protein_coding,...,chr1,HAVANA,69091,+,CCDS,ENST00000335137.3,OR4F5-001,KNOWN,None,protein_coding
131,None,182711,ENSE00003759020.1,1,start_codon,0,ENSG00000279928.1,FO538757.2,KNOWN,protein_coding,...,chr1,ENSEMBL,182709,+,appris_principal_1,ENST00000624431.1,FO538757.2-201,KNOWN,1,protein_coding
143,None,195411,ENSE00003758618.1,1,start_codon,0,ENSG00000279457.3,FO538757.1,KNOWN,protein_coding,...,chr1,ENSEMBL,195409,-,appris_alternative_2,ENST00000623834.3,FO538757.1-201,KNOWN,5,protein_coding
167,None,195411,ENSE00003755778.1,1,start_codon,0,ENSG00000279457.3,FO538757.1,KNOWN,protein_coding,...,chr1,ENSEMBL,195409,-,appris_principal_5,ENST00000623083.3,FO538757.1-202,KNOWN,5,protein_coding
193,None,200086,ENSE00003757911.1,1,start_codon,0,ENSG00000279457.3,FO538757.1,KNOWN,protein_coding,...,chr1,ENSEMBL,200084,-,appris_alternative_2,ENST00000624735.1,FO538757.1-203,KNOWN,5,protein_coding


In [6]:
start_codons['length'] = start_codons['end'].astype(int)-start_codons['start'].astype(int)+1
start_codons['length'].describe()

/auto/cmb-panasas2/skchoudh/software_frozen/anaconda27/envs/riboraptor/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


count    82648.000000
mean         2.990744
std          0.123860
min          1.000000
25%          3.000000
50%          3.000000
75%          3.000000
max          3.000000
Name: length, dtype: float64

In [7]:
start_codons[start_codons['length']==1]

,ccdsid,end,exon_id,exon_number,feature,frame,gene_id,gene_name,gene_status,gene_type,...,source,start,strand,tag,transcript_id,transcript_name,transcript_status,transcript_support_level,transcript_type,length
13738,None,6463072,ENSE00003657231.1,5,start_codon,0,ENSG00000215788.9,TNFRSF25,KNOWN,protein_coding,...,HAVANA,6463072,-,cds_end_NF,ENST00000481401.5,TNFRSF25-012,KNOWN,5,protein_coding,1
15068,CCDS88.1,7773442,ENSE00001474297.1,2,start_codon,1,ENSG00000049245.12,VAMP3,KNOWN,protein_coding,...,HAVANA,7773442,+,CCDS,ENST00000054666.10,VAMP3-001,KNOWN,1,protein_coding,1
34452,CCDS57978.1,23014045,ENSE00002623552.1,2,start_codon,1,ENSG00000227868.5,C1orf234,KNOWN,protein_coding,...,HAVANA,23014045,-,CCDS,ENST00000566855.3,C1orf234-001,KNOWN,2,protein_coding,1
55796,CCDS59194.1,36139819,ENSE00003490237.1,3,start_codon,1,ENSG00000054116.11,TRAPPC3,KNOWN,protein_coding,...,ENSEMBL,36139819,-,CCDS,ENST00000617904.4,TRAPPC3-203,KNOWN,2,protein_coding,1
55831,CCDS59194.1,36139819,ENSE00003490237.1,3,start_codon,1,ENSG00000054116.11,TRAPPC3,KNOWN,protein_coding,...,HAVANA,36139819,-,CCDS,ENST00000373163.5,TRAPPC3-005,KNOWN,3,protein_coding,1
55847,CCDS59194.1,36139819,ENSE00003490237.1,3,start_codon,1,ENSG00000054116.11,TRAPPC3,KNOWN,protein_coding,...,HAVANA,36139819,-,CCDS,ENST00000373162.5,TRAPPC3-004,KNOWN,2,protein_coding,1
75736,None,46665364,ENSE00003481921.1,2,start_codon,1,ENSG00000123472.12,ATPAF1,KNOWN,protein_coding,...,HAVANA,46665364,-,None,ENST00000529214.5,ATPAF1-009,KNOWN,2,nonsense_mediated_decay,1
75796,CCDS57998.1,46665364,ENSE00003481921.1,2,start_codon,1,ENSG00000123472.12,ATPAF1,KNOWN,protein_coding,...,HAVANA,46665364,-,CCDS,ENST00000532925.5,ATPAF1-008,KNOWN,2,protein_coding,1
94090,None,69716218,ENSE00001453997.1,3,start_codon,0,ENSG00000033122.18,LRRC7,KNOWN,protein_coding,...,HAVANA,69716218,+,appris_alternative_2,ENST00000310961.9,LRRC7-002,KNOWN,5,protein_coding,1
94149,None,69678381,ENSE00001756600.1,2,start_codon,1,ENSG00000033122.18,LRRC7,KNOWN,protein_coding,...,HAVANA,69678381,+,basic,ENST00000370958.5,LRRC7-003,KNOWN,1,protein_coding,1


In [9]:
start_codons['seq'] = start_codons.apply(get_seq, axis=1)

AssertionError: occurred at index 56